In [1]:
import time
import pickle
import cv2 as cv
import numpy as np
import pandas as pd
from glob import glob
from config import efficientNet_config
from models import efficientNetV2B0_model

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
model = efficientNetV2B0_model()
model.load_weights('../model/202303/20230329.weights')

2023-05-03 14:35:36.086884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 14:35:36.092090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 14:35:36.093051: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 14:35:36.093910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 5,859,985
Non-trainable params: 60,608
_________________________________________________________________


In [3]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']

In [4]:
testdata = "test_20210407"
test_dir      = '../dataset_test_everyframes/{}/'.format(testdata)
## everyframes
img_paths = sorted(glob(test_dir + "*.png"))

start_time = time.time()
s_time     = time.time()
loop_times = []

pred_results = []
img_names = []
for i,img_path in enumerate(img_paths):
    # imege
    img_name  = img_path.split("/")[-1]
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))     # resize to (224,224)
    
    
    pred_result = model.predict(np.array([img_array]))
    pred_result = np.where(pred_result > 0.5, 1, 0)
    
    if pred_result[0][0]!=0:
        print(pred_result[0][0])
        pred_results.append(pred_result[0][0])
        img_names.append(img_name)

    # 計算每張圖片計算時間
    temp_time = time.time()
    loop_times.append(temp_time-s_time)
    s_time = temp_time

end_time = time.time()

print(end_time-start_time)

2023-05-03 14:35:40.122446: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 17ms/step
1
1/1 [=======================

In [5]:
print("Best time on = {:.6f}".format(min(sorted(loop_times)[0:-50])))
print("Worst time on = {:.6f}".format(max(sorted(loop_times)[0:-50])))
print("Avg time on = {:.6f}".format(sum(sorted(loop_times)[0:-50])/len(sorted(loop_times)[0:-50])))

Best time on = 0.040632
Worst time on = 0.049138
Avg time on = 0.042885


In [6]:
stage1_everyframes_result = pd.DataFrame({"stage1_pred_result":pred_results,"img_name":img_names})

In [7]:
with open('../dataset_test_everyframes/{}_stage1_everyframes_result_20230430.pk'.format(testdata), 'wb') as f:
    pickle.dump(stage1_everyframes_result, f)